# **Análisis y Predicción de Precios de Autos (Core2)**

In [2]:
# Conectamos colab con drive.
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Importo librerias
import pandas as pd
import numpy as np

**Paso 1: Cargar el dataset**

In [4]:
df = pd.read_csv('/content/drive/MyDrive/Bootcamp-ML/Cores/Core2_KNN_arbolDecision/Automobile_data.csv')


**Paso 2: Comprender la estructura general**

In [5]:
# Dimensiones del dataset
print("Filas y columnas:", df.shape)

Filas y columnas: (205, 26)


In [6]:
# Información general del dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  205 non-null    object 
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       205 non-null    object 
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

In [7]:
# Estadísticas básicas
df.describe().T.round()

,count,mean,std,min,25%,50%,75%,max
symboling,205.0,1.0,1.0,-2.0,0.0,1.0,2.0,3.0
wheel-base,205.0,99.0,6.0,87.0,94.0,97.0,102.0,121.0
length,205.0,174.0,12.0,141.0,166.0,173.0,183.0,208.0
width,205.0,66.0,2.0,60.0,64.0,66.0,67.0,72.0
height,205.0,54.0,2.0,48.0,52.0,54.0,56.0,60.0
curb-weight,205.0,2556.0,521.0,1488.0,2145.0,2414.0,2935.0,4066.0
engine-size,205.0,127.0,42.0,61.0,97.0,120.0,141.0,326.0
compression-ratio,205.0,10.0,4.0,7.0,9.0,9.0,9.0,23.0
city-mpg,205.0,25.0,7.0,13.0,19.0,24.0,30.0,49.0
highway-mpg,205.0,31.0,7.0,16.0,25.0,30.0,34.0,54.0


In [8]:
# Identifica las columnas
df.columns

Index(['symboling', 'normalized-losses', 'make', 'fuel-type', 'aspiration',
       'num-of-doors', 'body-style', 'drive-wheels', 'engine-location',
       'wheel-base', 'length', 'width', 'height', 'curb-weight', 'engine-type',
       'num-of-cylinders', 'engine-size', 'fuel-system', 'bore', 'stroke',
       'compression-ratio', 'horsepower', 'peak-rpm', 'city-mpg',
       'highway-mpg', 'price'],
      dtype='object')

In [9]:
df.head(10)

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450
5,2,?,audi,gas,std,two,sedan,fwd,front,99.8,...,136,mpfi,3.19,3.4,8.5,110,5500,19,25,15250
6,1,158,audi,gas,std,four,sedan,fwd,front,105.8,...,136,mpfi,3.19,3.4,8.5,110,5500,19,25,17710
7,1,?,audi,gas,std,four,wagon,fwd,front,105.8,...,136,mpfi,3.19,3.4,8.5,110,5500,19,25,18920
8,1,158,audi,gas,turbo,four,sedan,fwd,front,105.8,...,131,mpfi,3.13,3.4,8.3,140,5500,17,20,23875
9,0,?,audi,gas,turbo,two,hatchback,4wd,front,99.5,...,131,mpfi,3.13,3.4,7.0,160,5500,16,22,?


In [10]:
# Reemplazar los "?" por NaN
df['bore']=df['bore'].replace("?", np.nan)
df['stroke']=df['stroke'].replace("?", np.nan)

# Convertir a tipo float
df['bore'] = df['bore'].astype(float)
df['stroke'] = df['stroke'].astype(float)

# Imputar valores faltantes con la mediana
df['bore']=df['bore'].fillna(df['bore'].median())
df['stroke']=df['stroke'].fillna(df['stroke'].median())

In [11]:
conteo_por_columna = (df.astype(str) == "?").sum()
print(conteo_por_columna)

symboling             0
normalized-losses    41
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  0
stroke                0
compression-ratio     0
horsepower            2
peak-rpm              2
city-mpg              0
highway-mpg           0
price                 4
dtype: int64


In [12]:
# Reemplazar los "?" por NaN
df['horsepower']=df['horsepower'].replace("?", np.nan)
df['peak-rpm']=df['peak-rpm'].replace("?", np.nan)

# Convertir a tipo float
df['horsepower'] = df['horsepower'].astype(float)
df['peak-rpm'] = df['peak-rpm'].astype(float)

# Imputar valores faltantes con la mediana
df['horsepower']=df['horsepower'].fillna(df['horsepower'].median())
df['peak-rpm']=df['peak-rpm'].fillna(df['peak-rpm'].median())

In [13]:
conteo_por_columna = (df.astype(str) == "?").sum()
print(conteo_por_columna)

symboling             0
normalized-losses    41
make                  0
fuel-type             0
aspiration            0
num-of-doors          2
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  0
stroke                0
compression-ratio     0
horsepower            0
peak-rpm              0
city-mpg              0
highway-mpg           0
price                 4
dtype: int64


In [14]:
# Reemplazar los "?" por NaN
df['num-of-doors']=df['num-of-doors'].replace("?", np.nan)
df['price']=df['price'].replace("?", np.nan)

df['num-of-doors'].value_counts()

,count
num-of-doors,
four,114
two,89


In [15]:
# Convertir a tipo float
#df['num-of-doors'] = df['num-of-doors'].astype(float)
df['price'] = df['price'].astype(float)

# Imputar valores faltantes con la mediana
#df['num-of-doors']= df['num-of-doors'].fillna(df['num-of-doors'].mode())
df['price']= df['price'].fillna(df['price'].mode())

In [16]:
conteo_por_columna = (df.astype(str) == "?").sum()
print(conteo_por_columna)

symboling             0
normalized-losses    41
make                  0
fuel-type             0
aspiration            0
num-of-doors          0
body-style            0
drive-wheels          0
engine-location       0
wheel-base            0
length                0
width                 0
height                0
curb-weight           0
engine-type           0
num-of-cylinders      0
engine-size           0
fuel-system           0
bore                  0
stroke                0
compression-ratio     0
horsepower            0
peak-rpm              0
city-mpg              0
highway-mpg           0
price                 0
dtype: int64


In [17]:
dic={
    'four':4,
    'two':2
}

df['num-of-doors'] = df['num-of-doors'].map(dic)



# Verificar resultado
print(df['num-of-doors'].unique())
print(df['num-of-doors'].isnull().sum())
print(df['num-of-doors'].dtype)

[ 2.  4. nan]
2
float64


In [18]:
# Llenar los valores faltantes con la moda
moda = df['num-of-doors'].mode()[0]
df['num-of-doors'].fillna(moda, inplace=True)

/tmp/ipython-input-18-1294783161.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['num-of-doors'].fillna(moda, inplace=True)


In [19]:
# Verificar resultado
print(df['num-of-doors'].unique())
print(df['num-of-doors'].isnull().sum())
print(df['num-of-doors'].dtype)

[2. 4.]
0
float64


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  205 non-null    object 
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       205 non-null    float64
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

In [21]:
# Reemplazar los "?" por NaN
df['normalized-losses']= df['normalized-losses'].replace("?", np.nan)


# Convertir a tipo float
df['normalized-losses'] = df['normalized-losses'].astype(float)


# Imputar valores faltantes con la mediana
df['normalized-losses']=df['normalized-losses'].fillna(df['normalized-losses'].median())


In [22]:
conteo_por_columna = (df.astype(str) == "?").sum()
print(conteo_por_columna)

symboling            0
normalized-losses    0
make                 0
fuel-type            0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
dtype: int64


In [23]:
print(df.isnull().sum())

symboling            0
normalized-losses    0
make                 0
fuel-type            0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                3
dtype: int64


In [24]:
# Verificar resultado
print(df['price'].unique())
print(df['price'].isnull().sum())
print(df['price'].dtype)

[13495. 16500. 13950. 17450. 15250. 17710. 18920. 23875.  8845. 16430.
 16925. 20970. 21105. 24565. 30760. 41315. 36880.  5151.  6295.  6575.
  5572.  6377.  7957.  6229.  6692.  7609.  8558.  8921. 12964.  6479.
  6855.  5399.  6529.  7129.  7295.  7895.  9095. 10295. 12945. 10345.
  6785.    nan 11048. 32250. 35550. 36000.  5195.  6095.  6795.  6695.
  7395. 10945. 11845. 13645. 15645.  8495. 10595. 10245. 10795. 11245.
 18280. 18344. 25552. 28248. 28176. 31600. 34184. 35056. 40960. 45400.
 16503.  5389.  6189.  6669.  7689.  9959.  8499. 12629. 14869. 14489.
  6989.  8189.  9279.  5499.  7099.  6649.  6849.  7349.  7299.  7799.
  7499.  7999.  8249.  8949.  9549. 13499. 14399. 17199. 19699. 18399.
 11900. 13200. 12440. 13860. 15580. 16900. 16695. 17075. 16630. 17950.
 18150. 12764. 22018. 32528. 34028. 37028.  9295.  9895. 11850. 12170.
 15040. 15510. 18620.  5118.  7053.  7603.  7126.  7775.  9960.  9233.
 11259.  7463. 10198.  8013. 11694.  5348.  6338.  6488.  6918.  7898.
  8778

In [25]:
# Llenar los valores faltantes con la moda
moda = df['price'].mode()[0]
df['price'].fillna(moda, inplace=True)

/tmp/ipython-input-25-3681530645.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['price'].fillna(moda, inplace=True)


In [26]:
print(df.isnull().sum())

symboling            0
normalized-losses    0
make                 0
fuel-type            0
aspiration           0
num-of-doors         0
body-style           0
drive-wheels         0
engine-location      0
wheel-base           0
length               0
width                0
height               0
curb-weight          0
engine-type          0
num-of-cylinders     0
engine-size          0
fuel-system          0
bore                 0
stroke               0
compression-ratio    0
horsepower           0
peak-rpm             0
city-mpg             0
highway-mpg          0
price                0
dtype: int64


**Exportar Dataset Limpio**

In [27]:
path = '/content/drive/MyDrive/Bootcamp-ML/Cores/Core2_KNN_arbolDecision/core2_autos_clean.csv'
df.to_csv(path, index=False)

# 📶<font color="green">**Modelos de Regresión**</font>


In [78]:
# importaciones
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 26 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   symboling          205 non-null    int64  
 1   normalized-losses  205 non-null    float64
 2   make               205 non-null    object 
 3   fuel-type          205 non-null    object 
 4   aspiration         205 non-null    object 
 5   num-of-doors       205 non-null    float64
 6   body-style         205 non-null    object 
 7   drive-wheels       205 non-null    object 
 8   engine-location    205 non-null    object 
 9   wheel-base         205 non-null    float64
 10  length             205 non-null    float64
 11  width              205 non-null    float64
 12  height             205 non-null    float64
 13  curb-weight        205 non-null    int64  
 14  engine-type        205 non-null    object 
 15  num-of-cylinders   205 non-null    object 
 16  engine-size        205 non

#⏩ <font color="green">**Machine Learning = Regresión Lineal**</font>


In [30]:
# Definir features y target.
X = df.drop(columns="price")
y = df["price"]

In [31]:
# Dividir en train y test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [32]:
# Agrupar tipos de features.
num_cols = ['symboling','normalized-losses',
       'wheel-base', 'length', 'width', 'height','num-of-doors', 'curb-weight',  'engine-size','bore', 'stroke','compression-ratio', 'horsepower', 'peak-rpm','city-mpg','highway-mpg']

nom_cols = ['make', 'fuel-type', 'aspiration', 'body-style', 'drive-wheels', 'engine-location','engine-type','num-of-cylinders','fuel-system' ]

In [33]:
preprocessorRL = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_cols),
        #("ord", OrdinalEncoder(categories=[["Third", "Second", "First"]]), ord_cols),  en este core no hay ordinales
        ("nom", OneHotEncoder(handle_unknown="ignore", sparse_output=False), nom_cols),
    ]
)

In [34]:
# Pipeline con regresión lineal
pipelineRL = Pipeline(steps=[
    ("preprocessing", preprocessorRL),
    ("regressor", LinearRegression())
])


In [35]:
# Entrenar modelo
pipelineRL.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['symboling',
                                                   'normalized-losses',
                                                   'wheel-base', 'length',
                                                   'width', 'height',
                                                   'num-of-doors',
                                                   'curb-weight', 'engine-size',
                                                   'bore', 'stroke',
                                                   'compression-ratio',
                                                   'horsepower', 'peak-rpm',
                                                   'city-mpg', 'highway-mpg']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['make', 'fuel-type',
                                                   'aspiration', 'body-style',
                                                   'drive-wheels',
                                                   'engine-location',
                                                   'engine-type',
                                                   'num-of-cylinders',
                                                   'fuel-system'])])),
                ('regressor', LinearRegression())])

In [36]:
y_pred = pipelineRL.predict(X_test)

In [37]:
r2 = r2_score(y_test, y_pred)
print(f"R²: {r2:.4f}")

R²: 0.8434


In [38]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE: ${rmse:,.2f}")

RMSE: $3,516.60


# ⏩ <font color="green">**Machine Learning = Modelo Arbol**</font>


In [39]:
# Definir features y target.
X = df.drop(columns="price")
y = df["price"]

In [40]:
# Dividir en train y test.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [79]:
# Agrupar tipos de features.
num_cols = ['symboling','normalized-losses',
       'wheel-base', 'length', 'width', 'height','num-of-doors', 'curb-weight',  'engine-size','bore', 'stroke','compression-ratio', 'horsepower', 'peak-rpm','city-mpg','highway-mpg']

nom_cols = ['make', 'fuel-type', 'aspiration', 'body-style', 'drive-wheels', 'engine-location','engine-type','num-of-cylinders','fuel-system' ]

In [80]:
# Definir transformadores.
preprocessorarbol = ColumnTransformer(transformers=[
     ("num", "passthrough", num_cols),  # Árboles no necesitan escalar
   # ("ord", OrdinalEncoder(categories=[["Third", "Second", "First"]]), ord_cols),
    ("nom", OneHotEncoder(handle_unknown='ignore'), nom_cols),
])

In [81]:
# Crear pipeline.
pipelinearbol = Pipeline(steps=[
    ("preprocessing", preprocessorarbol),
    ("model", DecisionTreeRegressor(max_depth=8, random_state=42))
])

In [44]:
pipelinearbol

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['symboling',
                                                   'normalized-losses',
                                                   'wheel-base', 'length',
                                                   'width', 'height',
                                                   'num-of-doors',
                                                   'curb-weight', 'engine-size',
                                                   'bore', 'stroke',
                                                   'compression-ratio',
                                                   'horsepower', 'peak-rpm',
                                                   'city-mpg', 'highway-mpg']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['make', 'fuel-type',
                                                   'aspiration', 'body-style',
                                                   'drive-wheels',
                                                   'engine-location',
                                                   'engine-type',
                                                   'num-of-cylinders',
                                                   'fuel-system'])])),
                ('classifier', DecisionTreeClassifier(random_state=42))])

In [82]:
# Entrenar modelo.
pipelinearbol.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', 'passthrough',
                                                  ['symboling',
                                                   'normalized-losses',
                                                   'wheel-base', 'length',
                                                   'width', 'height',
                                                   'num-of-doors',
                                                   'curb-weight', 'engine-size',
                                                   'bore', 'stroke',
                                                   'compression-ratio',
                                                   'horsepower', 'peak-rpm',
                                                   'city-mpg', 'highway-mpg']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['make', 'fuel-type',
                                                   'aspiration', 'body-style',
                                                   'drive-wheels',
                                                   'engine-location',
                                                   'engine-type',
                                                   'num-of-cylinders',
                                                   'fuel-system'])])),
                ('model', DecisionTreeRegressor(max_depth=8, random_state=42))])

In [83]:
# Predecir y evaluar.
y_pred = pipelinearbol.predict(X_test)

In [84]:

y_pred_tree = pipelinearbol.predict(X_test)
r2_tree = r2_score(y_test, y_pred_tree)
rmse_tree = np.sqrt(mean_squared_error(y_test, y_pred_tree))

In [85]:
print(f'Error Cuadrático Medio: {mse}')
print(f'R-cuadrado: {r2}')

Error Cuadrático Medio: 16311722.644986449
R-cuadrado: 0.7934482099537583


In [86]:
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE: ${rmse:,.2f}")

RMSE: $2,841.62


# ⏩ <font color="green">**Machine Learning = KNN**</font>



In [70]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.neighbors import KNeighborsClassifier # KNN
from sklearn.neighbors import KNeighborsRegressor # KNN
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


In [51]:
# Separar variables predictoras y objetivo
X = df.drop(columns="price")
y = df["price"]

In [52]:
# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [53]:
# Definir transformadores.
num_cols = ['symboling','normalized-losses','wheel-base', 'length', 'width', 'height','num-of-doors', 'curb-weight',  'engine-size','bore','stroke','compression-ratio', 'horsepower', 'peak-rpm','city-mpg','highway-mpg']
nom_cols = ['make', 'fuel-type', 'aspiration', 'body-style', 'drive-wheels', 'engine-location','engine-type','num-of-cylinders','fuel-system' ]

In [54]:
# Definir transformadores.
preprocessorknn = ColumnTransformer(transformers=[
     ("num", StandardScaler(), num_cols),
     ("nom", OneHotEncoder(handle_unknown='ignore'), nom_cols),
])

In [104]:
# Pipeline.
pipelineknn = Pipeline(steps=[
    ("preprocessing", preprocessorknn),
    ("regressor", KNeighborsRegressor(n_neighbors=2))
])

In [105]:
# Entrenar el modelo
pipelineknn.fit(X_train, y_train)

Pipeline(steps=[('preprocessing',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['symboling',
                                                   'normalized-losses',
                                                   'wheel-base', 'length',
                                                   'width', 'height',
                                                   'num-of-doors',
                                                   'curb-weight', 'engine-size',
                                                   'bore', 'stroke',
                                                   'compression-ratio',
                                                   'horsepower', 'peak-rpm',
                                                   'city-mpg', 'highway-mpg']),
                                                 ('nom',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['make', 'fuel-type',
                                                   'aspiration', 'body-style',
                                                   'drive-wheels',
                                                   'engine-location',
                                                   'engine-type',
                                                   'num-of-cylinders',
                                                   'fuel-system'])])),
                ('regressor', KNeighborsRegressor(n_neighbors=2))])

In [106]:
# Hacer predicciones
y_pred = pipelineknn.predict(X_test)

In [107]:
# Evaluar el modelo
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Error Cuadrático Medio: {mse}')
print(f'R-cuadrado: {r2}')

Error Cuadrático Medio: 9843027.341463415
R-cuadrado: 0.8753598892586457


In [59]:
from sklearn.metrics import mean_absolute_error

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)

print(f"RMSE: ${rmse:,.2f}")

RMSE: $4,038.78



# ✅ <font color="blue">**Benchmarking y Comparación de Modelos**</font>
1. **Regresión Lineal**
* R2= 0.8434
* RMSE(USD) $3516.60

* Mejor rendimiento general de los 3 modelos aplicados

2. **Arbol de Decision**
* R2 = 0.7982
* RMSE (USD) $ 2800.41

* Buen rendimiento, similar a Regresión Lineal

3. **KNN (k=3)**
* R2 = 0.7934
* RMSE (USD) = $4038.70
* error mas alto, y mas preciso que con otros valores de k


# ✅ <font color="blue">**Interpretación**</font>


**Regresión Lineal** es el algoritmo más preciso en este caso, con un tienen R2 de (0.8434). Sin embargo, el algoritmo **arbol de decisión** tiene el menor RMSE, lo que sugiere una mayor precisión en las predicciones.

# ✅ <font color="blue">**Tareas Adicionales y Desafíos**</font>


# **Gridsearch**

In [60]:
from sklearn.model_selection import GridSearchCV

# **Optimizadores para KNN**

In [87]:
# Preprocesador.
preprocessor_knnGS = ColumnTransformer(transformers=[
    ("num", StandardScaler(), num_cols)
])

# Model.
knn_pipelineGS = Pipeline([
    ("pp", preprocessor_knnGS),
    ("model", KNeighborsRegressor()) #no se define el nro de vecinos
])

In [100]:
# Optimizacion de hiperparametros.
knn_params = {
    "model__n_neighbors": [2,4,8,10]

    }

knn_grid = GridSearchCV(knn_pipelineGS, knn_params, cv=5, scoring="r2")
knn_grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pp',
                                        ColumnTransformer(transformers=[('num',
                                                                         StandardScaler(),
                                                                         ['symboling',
                                                                          'normalized-losses',
                                                                          'wheel-base',
                                                                          'length',
                                                                          'width',
                                                                          'height',
                                                                          'num-of-doors',
                                                                          'curb-weight',
                                                                          'engine-size',
                                                                          'bore',
                                                                          'stroke',
                                                                          'compression-ratio',
                                                                          'horsepower',
                                                                          'peak-rpm',
                                                                          'city-mpg',
                                                                          'highway-mpg'])])),
                                       ('model', KNeighborsRegressor())]),
             param_grid={'model__n_neighbors': [2, 4, 8, 10]}, scoring='r2')

In [101]:
# Evaluación.
knn_best = knn_grid.best_estimator_ #prediccion con la mejor estimador

In [102]:
knn_best

Pipeline(steps=[('pp',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['symboling',
                                                   'normalized-losses',
                                                   'wheel-base', 'length',
                                                   'width', 'height',
                                                   'num-of-doors',
                                                   'curb-weight', 'engine-size',
                                                   'bore', 'stroke',
                                                   'compression-ratio',
                                                   'horsepower', 'peak-rpm',
                                                   'city-mpg',
                                                   'highway-mpg'])])),
                ('model', KNeighborsRegressor(n_neighbors=2))])

In [103]:
y_pred_knnGS = knn_best.predict(X_test)

print("KNN Regressor")
print("Mejores parámetros:", knn_grid.best_params_)
print("R²:", r2_score(y_test, y_pred_knnGS))

KNN Regressor
Mejores parámetros: {'model__n_neighbors': 2}
R²: 0.6575854941130601


***RANDOM FOREST***

In [68]:
from sklearn.model_selection import GridSearchCV

In [71]:
# Preprocesador.
preprocessor_forestGS = ColumnTransformer(transformers=[
    ("num", "passthrough", num_cols)
])

# Model.
pipeline_forestGS = Pipeline([
    ("pp", preprocessor_forestGS),
    ("model", RandomForestRegressor(random_state=42))
])

In [72]:
# Optimizacion de hiperparametros.
forest_params = {
    "model__n_estimators": [50, 100, 200] # El nombre debe coincidir con el nombre del modelo.
}

forest_grid = GridSearchCV(pipeline_forestGS, forest_params, cv=3, scoring="r2")
forest_grid.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('pp',
                                        ColumnTransformer(transformers=[('num',
                                                                         'passthrough',
                                                                         ['symboling',
                                                                          'normalized-losses',
                                                                          'wheel-base',
                                                                          'length',
                                                                          'width',
                                                                          'height',
                                                                          'num-of-doors',
                                                                          'curb-weight',
                                                                          'engine-size',
                                                                          'bore',
                                                                          'stroke',
                                                                          'compression-ratio',
                                                                          'horsepower',
                                                                          'peak-rpm',
                                                                          'city-mpg',
                                                                          'highway-mpg'])])),
                                       ('model',
                                        RandomForestRegressor(random_state=42))]),
             param_grid={'model__n_estimators': [50, 100, 200]}, scoring='r2')

In [73]:
# Evaluación.
forest_best = forest_grid.best_estimator_
y_pred_forestGS = forest_best.predict(X_test)


In [75]:
print("Forest Regressor")
print("Mejores parámetros:", forest_grid.best_params_)
print("R²:", r2_score(y_test, y_pred_forestGS))

Forest Regressor
Mejores parámetros: {'model__n_estimators': 200}
R²: 0.9243776316665758


# **Resumen: GridSearchCV para ajustar los hiperparámetros de KNN y Random Forest**

1. Para **KNN**, el mejor parámetro encontrado fue un n_neighbors de 2 y R2 de 0.6576, resultado que empeora al reducir el K, por lo tanto, es un modelo sensible al sobreajuste con pocos vecinos.
2. Para **Random Forest Regressor**, el mejor parámetro encontrado es un n_estimator de 200, y un R2 de 0.9244. ***Es el algoritmo mas preciso de todos***.